In [75]:
using ITensors
using CUDA

In [76]:
CUDA.devices()

CUDA.DeviceIterator() for 2 devices:
0. Tesla V100-SXM2-32GB
1. Tesla V100-SXM2-32GB

In [77]:
#monitoring multiples gpu functions:

function memory_info_all_gpus(print_info = true)
    
    percentages = []

    scale = 1/(1024^3) #converty bytes to GB
    for (i, dev) in enumerate(CUDA.NVML.devices())

        name = CUDA.NVML.name(dev) 
        mem_info = CUDA.NVML.memory_info(dev)
        total = round(mem_info.total*scale, sigdigits=4)
        used = round(mem_info.used*scale, sigdigits=4)
        free = round(mem_info.free*scale, sigdigits=4)
        percentage= round(used*100/total, sigdigits=4)
        
        print_info ? println("$name #$i memory usage: $percentage % ($used GB/ $total GB)" ) : nothing
        
        append!(percentages, percentage)
    end
    
    return percentages
end

function clean_all_gpus(Deep_cleaning = false)
    for i=reverse(0:length(CUDA.devices()) - 1)
        global current_gpu = i
        CUDA.device!(current_gpu)
        Deep_cleaning ? GC.gc(true) : nothing #This could be very slow.
        CUDA.reclaim()
    end
end

clean_all_gpus (generic function with 2 methods)

In [100]:
@time memory_info_all_gpus()
@time CUDA.memory_status() #It just print the one that is in use.

Tesla V100-SXM2-32GB #1 memory usage: 59.25 % (18.96 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 92.84 % (29.71 GB/ 32.0 GB)
  0.000156 seconds (72 allocations: 5.039 KiB)
Effective GPU memory usage: 58.91% (18.699 GiB/31.739 GiB)
Memory pool usage: 15.094 GiB (18.312 GiB reserved)
  0.000183 seconds (63 allocations: 4.070 KiB)


In [79]:
function Create_H_MPO(t,U, N, sites = []) 

    if length(sites) == 0
        sites = siteinds("Electron",N)    
    end
    
    os = OpSum() 
    for j=1:N-1 
        os += -t,"Cdagup",j,"Cup",j+1 
        os += -t,"Cdagup",j+1,"Cup",j
        os += -t,"Cdagdn",j,"Cdn",j+1 
        os += -t,"Cdagdn",j+1,"Cdn",j
    end 

    for j=1:N
        os += U,"Nup * Ndn",j
    end
    # Convert these terms to an MPO 
    H = MPO(os,sites)
    return H, sites
end

#Custom observer to measure the use of GPU:

mutable struct DemoObserver <: AbstractObserver
    energy_tol::Float64
    last_energy::Float64

    DemoObserver(energy_tol=0.0) = new(energy_tol,1000.0)
end

cleaning_in_each_sweep = false #defaultt

function ITensors.checkdone!(o::DemoObserver;kwargs...)
    
    memory_info_all_gpus() #Print GPU percentage of use. 
    
    # # What if we clean cache in each sweep?.
    
    if cleaning_in_each_sweep
        GC.gc(true) 
        CUDA.reclaim()
    end    
        
    sw = kwargs[:sweep]
    energy = kwargs[:energy]
    if abs(energy-o.last_energy)/abs(energy) < o.energy_tol
    println("Stopping DMRG after sweep $sw")
    return true
    end
    # Otherwise, update last_energy and keep going
    o.last_energy = energy
    return false
end

In [101]:
clean_all_gpus(true)
memory_info_all_gpus()

Tesla V100-SXM2-32GB #1 memory usage: 39.31 % (12.58 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 9.834 % (3.147 GB/ 32.0 GB)


2-element Vector{Any}:
 39.31
  9.834

In [84]:
N = 60
t = 1
U = 1

nsweeps = 100
maxdim = [1000] #maxdim - integer or array of integers specifying the maximum size allowed for the bond dimension or rank of the MPS being optimized
cutoff = [1E-10] #maxdim - integer or array of integers specifying the maximum size allowed for the bond dimension or rank of the MPS being optimized
# DMRG_observer = DMRGObserver(;energy_tol=10e-8, minsweeps=10, energy_type=Float64)
DMRG_observer = DemoObserver(10e-8)

DemoObserver(1.0e-7, 1000.0)

In [85]:
H, sites = Create_H_MPO(t,U, N) 
Initial_Guess = randomMPS(sites);
# @time energy_ground_state, psi_ground_state = dmrg(H,Initial_Guess; nsweeps, maxdim, cutoff, observer = DMRG_observer, outputlevel = 1) 

In [86]:
cleaning_in_each_sweep = false
H = NDTensors.cu(H)
Initial_Guess = NDTensors.cu(Initial_Guess)
@time energy_ground_state, psi_ground_state = dmrg(H,Initial_Guess; nsweeps, maxdim, cutoff, observer = DMRG_observer, outputlevel = 1) 

After sweep 1 energy=-61.40001643406533  maxlinkdim=16 maxerr=2.43E-16 time=0.423
Tesla V100-SXM2-32GB #1 memory usage: 62.56 % (20.02 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 2.022 % (0.6472 GB/ 32.0 GB)
After sweep 2 energy=-63.300510654556255  maxlinkdim=174 maxerr=1.00E-10 time=1.041
Tesla V100-SXM2-32GB #1 memory usage: 62.56 % (20.02 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 2.022 % (0.6472 GB/ 32.0 GB)
After sweep 3 energy=-63.52493609495797  maxlinkdim=284 maxerr=1.00E-10 time=2.378
Tesla V100-SXM2-32GB #1 memory usage: 62.56 % (20.02 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 2.022 % (0.6472 GB/ 32.0 GB)
After sweep 4 energy=-63.612046402727216  maxlinkdim=347 maxerr=1.00E-10 time=3.601
Tesla V100-SXM2-32GB #1 memory usage: 62.56 % (20.02 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 2.022 % (0.6472 GB/ 32.0 GB)
After sweep 5 energy=-63.65816472861775  maxlinkdim=403 maxerr=1.00E-10 time=5.057
Tesla V100-SXM2-32GB #1 memory usage: 65.41 % (20.93

(-63.71610198739973, MPS
[1] ((dim=4|id=161|"Link,l=1"), (dim=4|id=906|"Electron,Site,n=1"))
[2] ((dim=16|id=397|"Link,l=2"), (dim=4|id=303|"Electron,Site,n=2"), (dim=4|id=161|"Link,l=1"))
[3] ((dim=4|id=270|"Electron,Site,n=3"), (dim=63|id=85|"Link,l=3"), (dim=16|id=397|"Link,l=2"))
[4] ((dim=4|id=698|"Electron,Site,n=4"), (dim=166|id=280|"Link,l=4"), (dim=63|id=85|"Link,l=3"))
[5] ((dim=4|id=774|"Electron,Site,n=5"), (dim=285|id=949|"Link,l=5"), (dim=166|id=280|"Link,l=4"))
[6] ((dim=4|id=998|"Electron,Site,n=6"), (dim=375|id=603|"Link,l=6"), (dim=285|id=949|"Link,l=5"))
[7] ((dim=4|id=324|"Electron,Site,n=7"), (dim=488|id=983|"Link,l=7"), (dim=375|id=603|"Link,l=6"))
[8] ((dim=4|id=506|"Electron,Site,n=8"), (dim=551|id=556|"Link,l=8"), (dim=488|id=983|"Link,l=7"))
[9] ((dim=4|id=606|"Electron,Site,n=9"), (dim=649|id=527|"Link,l=9"), (dim=551|id=556|"Link,l=8"))
[10] ((dim=4|id=196|"Electron,Site,n=10"), (dim=791|id=113|"Link,l=10"), (dim=649|id=527|"Link,l=9"))
[11] ((dim=4|id=298|"

Here is clear that ITensors - DMRG does not works automatically with multiple GPUS. 

GPU 1 really needs 32 gbs in the latests sweeps? May be not, let's try to clean in each step, to see how actually needs by sweep.

In [48]:
clean_all_gpus(true)
memory_info_all_gpus()

Tesla V100-SXM2-32GB #1 memory usage: 6.906 % (2.21 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 1.76 % (0.5632 GB/ 32.0 GB)


2-element Vector{Any}:
 6.906
 1.76

In [49]:
cleaning_in_each_sweep = true
H = NDTensors.cu(H)
Initial_Guess = NDTensors.cu(Initial_Guess)
@time energy_ground_state, psi_ground_state = dmrg(H,Initial_Guess; nsweeps, maxdim, cutoff, observer = DMRG_observer, outputlevel = 1) 

After sweep 1 energy=-40.85257768482403  maxlinkdim=16 maxerr=3.35E-16 time=0.270
Tesla V100-SXM2-32GB #1 memory usage: 6.906 % (2.21 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 1.76 % (0.5632 GB/ 32.0 GB)
After sweep 2 energy=-41.99297741693292  maxlinkdim=169 maxerr=9.96E-11 time=0.671
Tesla V100-SXM2-32GB #1 memory usage: 27.9 % (8.928 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 1.76 % (0.5632 GB/ 32.0 GB)
After sweep 3 energy=-42.106681263678254  maxlinkdim=283 maxerr=1.00E-10 time=1.583
Tesla V100-SXM2-32GB #1 memory usage: 53.0 % (16.96 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 1.76 % (0.5632 GB/ 32.0 GB)
After sweep 4 energy=-42.16980224672787  maxlinkdim=397 maxerr=1.00E-10 time=2.671
Tesla V100-SXM2-32GB #1 memory usage: 54.47 % (17.43 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 1.76 % (0.5632 GB/ 32.0 GB)
After sweep 5 energy=-42.20914243337561  maxlinkdim=510 maxerr=1.00E-10 time=4.310
Tesla V100-SXM2-32GB #1 memory usage: 55.72 % (17.83 GB/ 32.

(-42.23687932832874, MPS
[1] ((dim=4|id=753|"Link,l=1"), (dim=4|id=15|"Electron,Site,n=1"))
[2] ((dim=16|id=613|"Link,l=2"), (dim=4|id=305|"Electron,Site,n=2"), (dim=4|id=753|"Link,l=1"))
[3] ((dim=4|id=536|"Electron,Site,n=3"), (dim=63|id=841|"Link,l=3"), (dim=16|id=613|"Link,l=2"))
[4] ((dim=4|id=191|"Electron,Site,n=4"), (dim=168|id=931|"Link,l=4"), (dim=63|id=841|"Link,l=3"))
[5] ((dim=4|id=411|"Electron,Site,n=5"), (dim=287|id=343|"Link,l=5"), (dim=168|id=931|"Link,l=4"))
[6] ((dim=4|id=114|"Electron,Site,n=6"), (dim=381|id=369|"Link,l=6"), (dim=287|id=343|"Link,l=5"))
[7] ((dim=4|id=568|"Electron,Site,n=7"), (dim=474|id=765|"Link,l=7"), (dim=381|id=369|"Link,l=6"))
[8] ((dim=4|id=620|"Electron,Site,n=8"), (dim=567|id=173|"Link,l=8"), (dim=474|id=765|"Link,l=7"))
[9] ((dim=4|id=886|"Electron,Site,n=9"), (dim=631|id=358|"Link,l=9"), (dim=567|id=173|"Link,l=8"))
[10] ((dim=4|id=851|"Electron,Site,n=10"), (dim=683|id=272|"Link,l=10"), (dim=631|id=358|"Link,l=9"))
[11] ((dim=4|id=779|

In [50]:
global current_gpu = 0 #the code always starts running with GPU 1.

function ITensors.checkdone!(o::DemoObserver;kwargs...)
        
    percentage = memory_info_all_gpus() #Print GPU percentage of use. 
    
    if percentage[current_gpu + 1] >= 90
        GC.gc(true) 
        CUDA.reclaim()
    end
    
    percentage = memory_info_all_gpus(false)
        
    if percentage[current_gpu + 1] >= 90
        global current_gpu = current_gpu + 1
        current_gpu == length(CUDA.devices()) ? clean_all_gpus(true) : CUDA.device!(current_gpu)
    end
        
    sw = kwargs[:sweep]
    energy = kwargs[:energy]
    if abs(energy-o.last_energy)/abs(energy) < o.energy_tol
    println("Stopping DMRG after sweep $sw")
    return true
    end
    # Otherwise, update last_energy and keep going
    o.last_energy = energy
    return false
end

In [51]:
clean_all_gpus(true)
memory_info_all_gpus()

Tesla V100-SXM2-32GB #1 memory usage: 11.1 % (3.553 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 1.76 % (0.5632 GB/ 32.0 GB)


2-element Vector{Any}:
 11.1
  1.76

In [52]:
H = NDTensors.cu(H)
Initial_Guess = NDTensors.cu(Initial_Guess)
@time energy_ground_state, psi_ground_state = dmrg(H,Initial_Guess; nsweeps, maxdim, cutoff, observer = DMRG_observer, outputlevel = 1) 

After sweep 1 energy=-40.85257768482403  maxlinkdim=16 maxerr=3.35E-16 time=0.296
Tesla V100-SXM2-32GB #1 memory usage: 11.1 % (3.553 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 1.76 % (0.5632 GB/ 32.0 GB)
After sweep 2 energy=-41.99297741693292  maxlinkdim=169 maxerr=9.96E-11 time=0.653
Tesla V100-SXM2-32GB #1 memory usage: 29.76 % (9.522 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 1.76 % (0.5632 GB/ 32.0 GB)
After sweep 3 energy=-42.106681263678254  maxlinkdim=283 maxerr=1.00E-10 time=1.588
Tesla V100-SXM2-32GB #1 memory usage: 53.0 % (16.96 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 1.76 % (0.5632 GB/ 32.0 GB)
After sweep 4 energy=-42.16980224672787  maxlinkdim=397 maxerr=1.00E-10 time=2.611
Tesla V100-SXM2-32GB #1 memory usage: 55.06 % (17.62 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 1.76 % (0.5632 GB/ 32.0 GB)
After sweep 5 energy=-42.20914243337561  maxlinkdim=510 maxerr=1.00E-10 time=4.250
Tesla V100-SXM2-32GB #1 memory usage: 57.88 % (18.52 GB/ 32

(-42.23687932832874, MPS
[1] ((dim=4|id=509|"Link,l=1"), (dim=4|id=15|"Electron,Site,n=1"))
[2] ((dim=16|id=249|"Link,l=2"), (dim=4|id=305|"Electron,Site,n=2"), (dim=4|id=509|"Link,l=1"))
[3] ((dim=4|id=536|"Electron,Site,n=3"), (dim=63|id=562|"Link,l=3"), (dim=16|id=249|"Link,l=2"))
[4] ((dim=4|id=191|"Electron,Site,n=4"), (dim=168|id=229|"Link,l=4"), (dim=63|id=562|"Link,l=3"))
[5] ((dim=4|id=411|"Electron,Site,n=5"), (dim=287|id=353|"Link,l=5"), (dim=168|id=229|"Link,l=4"))
[6] ((dim=4|id=114|"Electron,Site,n=6"), (dim=381|id=191|"Link,l=6"), (dim=287|id=353|"Link,l=5"))
[7] ((dim=4|id=568|"Electron,Site,n=7"), (dim=474|id=280|"Link,l=7"), (dim=381|id=191|"Link,l=6"))
[8] ((dim=4|id=620|"Electron,Site,n=8"), (dim=567|id=139|"Link,l=8"), (dim=474|id=280|"Link,l=7"))
[9] ((dim=4|id=886|"Electron,Site,n=9"), (dim=631|id=841|"Link,l=9"), (dim=567|id=139|"Link,l=8"))
[10] ((dim=4|id=851|"Electron,Site,n=10"), (dim=683|id=682|"Link,l=10"), (dim=631|id=841|"Link,l=9"))
[11] ((dim=4|id=779|

In [53]:
global current_gpu = 0 #the code always starts running with GPU 1.
CUDA.device!(current_gpu)

function ITensors.checkdone!(o::DemoObserver;kwargs...)
        
    percentage = memory_info_all_gpus() #Print GPU percentage of use. 
            
    if percentage[current_gpu + 1] >= 90
        global current_gpu = current_gpu + 1
        current_gpu == length(CUDA.devices()) ? clean_all_gpus(true) : CUDA.device!(current_gpu)
    end
        
    sw = kwargs[:sweep]
    energy = kwargs[:energy]
    if abs(energy-o.last_energy)/abs(energy) < o.energy_tol
    println("Stopping DMRG after sweep $sw")
    return true
    end
    # Otherwise, update last_energy and keep going
    o.last_energy = energy
    return false
end

In [54]:
H = NDTensors.cu(H)
Initial_Guess = NDTensors.cu(Initial_Guess)
@time energy_ground_state, psi_ground_state = dmrg(H,Initial_Guess; nsweeps, maxdim, cutoff, observer = DMRG_observer, outputlevel = 1) 

After sweep 1 energy=-40.85257768482403  maxlinkdim=16 maxerr=3.35E-16 time=0.276
Tesla V100-SXM2-32GB #1 memory usage: 18.62 % (5.96 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 1.76 % (0.5632 GB/ 32.0 GB)
After sweep 2 energy=-41.99297741693292  maxlinkdim=169 maxerr=9.96E-11 time=0.660
Tesla V100-SXM2-32GB #1 memory usage: 35.31 % (11.3 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 1.76 % (0.5632 GB/ 32.0 GB)
After sweep 3 energy=-42.106681263678254  maxlinkdim=283 maxerr=1.00E-10 time=1.579
Tesla V100-SXM2-32GB #1 memory usage: 53.69 % (17.18 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 1.76 % (0.5632 GB/ 32.0 GB)
After sweep 4 energy=-42.16980224672787  maxlinkdim=397 maxerr=1.00E-10 time=2.601
Tesla V100-SXM2-32GB #1 memory usage: 55.94 % (17.9 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 1.76 % (0.5632 GB/ 32.0 GB)
After sweep 5 energy=-42.20914243337561  maxlinkdim=510 maxerr=1.00E-10 time=4.263
Tesla V100-SXM2-32GB #1 memory usage: 60.22 % (19.27 GB/ 32.

(-42.23687932832874, MPS
[1] ((dim=4|id=647|"Link,l=1"), (dim=4|id=15|"Electron,Site,n=1"))
[2] ((dim=16|id=57|"Link,l=2"), (dim=4|id=305|"Electron,Site,n=2"), (dim=4|id=647|"Link,l=1"))
[3] ((dim=4|id=536|"Electron,Site,n=3"), (dim=63|id=345|"Link,l=3"), (dim=16|id=57|"Link,l=2"))
[4] ((dim=4|id=191|"Electron,Site,n=4"), (dim=168|id=203|"Link,l=4"), (dim=63|id=345|"Link,l=3"))
[5] ((dim=4|id=411|"Electron,Site,n=5"), (dim=287|id=557|"Link,l=5"), (dim=168|id=203|"Link,l=4"))
[6] ((dim=4|id=114|"Electron,Site,n=6"), (dim=381|id=147|"Link,l=6"), (dim=287|id=557|"Link,l=5"))
[7] ((dim=4|id=568|"Electron,Site,n=7"), (dim=474|id=609|"Link,l=7"), (dim=381|id=147|"Link,l=6"))
[8] ((dim=4|id=620|"Electron,Site,n=8"), (dim=567|id=790|"Link,l=8"), (dim=474|id=609|"Link,l=7"))
[9] ((dim=4|id=886|"Electron,Site,n=9"), (dim=631|id=83|"Link,l=9"), (dim=567|id=790|"Link,l=8"))
[10] ((dim=4|id=851|"Electron,Site,n=10"), (dim=683|id=982|"Link,l=10"), (dim=631|id=83|"Link,l=9"))
[11] ((dim=4|id=779|"Ele

In [96]:
clean_all_gpus(true)
memory_info_all_gpus()

Tesla V100-SXM2-32GB #1 memory usage: 33.38 % (10.68 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 9.834 % (3.147 GB/ 32.0 GB)


2-element Vector{Any}:
 33.38
  9.834

In [97]:
#Final version ? 

global current_gpu = 0 #the code always starts running with GPU 1.
CUDA.device!(current_gpu)

function ITensors.checkdone!(o::DemoObserver;kwargs...)
        
    percentage = memory_info_all_gpus() #Print GPU percentage of use. 
    # # What if we clean but not very deep in each sweep?.
    
    CUDA.reclaim() #It is the faster way to clean.
    percentage = memory_info_all_gpus(false) #Print GPU percentage of use. 
    if percentage[current_gpu + 1] >= 80
        global current_gpu = current_gpu + 1
        current_gpu == length(CUDA.devices()) ? clean_all_gpus() : CUDA.device!(current_gpu) #May be cleaning more deeply is good?
    end    
        
    sw = kwargs[:sweep]
    energy = kwargs[:energy]
    if abs(energy-o.last_energy)/abs(energy) < o.energy_tol
    println("Stopping DMRG after sweep $sw")
    return true
    end
    # Otherwise, update last_energy and keep going
    o.last_energy = energy
    return false
end

In [98]:
H = NDTensors.cu(H)
Initial_Guess = NDTensors.cu(Initial_Guess)
@time energy_ground_state, psi_ground_state = dmrg(H,Initial_Guess; nsweeps, maxdim, cutoff, observer = DMRG_observer, outputlevel = 1) 

After sweep 1 energy=-61.40001643406533  maxlinkdim=16 maxerr=2.43E-16 time=0.459
Tesla V100-SXM2-32GB #1 memory usage: 33.38 % (10.68 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 9.834 % (3.147 GB/ 32.0 GB)
After sweep 2 energy=-63.300510654556255  maxlinkdim=174 maxerr=1.00E-10 time=1.156
Tesla V100-SXM2-32GB #1 memory usage: 55.06 % (17.62 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 9.834 % (3.147 GB/ 32.0 GB)
After sweep 3 energy=-63.52493609495797  maxlinkdim=284 maxerr=1.00E-10 time=2.449
Tesla V100-SXM2-32GB #1 memory usage: 56.5 % (18.08 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 9.834 % (3.147 GB/ 32.0 GB)
After sweep 4 energy=-63.612046402727216  maxlinkdim=347 maxerr=1.00E-10 time=3.674
Tesla V100-SXM2-32GB #1 memory usage: 59.44 % (19.02 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 9.834 % (3.147 GB/ 32.0 GB)
After sweep 5 energy=-63.65816472861775  maxlinkdim=403 maxerr=1.00E-10 time=5.096
Tesla V100-SXM2-32GB #1 memory usage: 63.75 % (20.4 GB/ 3

(-63.71610198739973, MPS
[1] ((dim=4|id=768|"Link,l=1"), (dim=4|id=906|"Electron,Site,n=1"))
[2] ((dim=16|id=770|"Link,l=2"), (dim=4|id=303|"Electron,Site,n=2"), (dim=4|id=768|"Link,l=1"))
[3] ((dim=4|id=270|"Electron,Site,n=3"), (dim=63|id=789|"Link,l=3"), (dim=16|id=770|"Link,l=2"))
[4] ((dim=4|id=698|"Electron,Site,n=4"), (dim=166|id=133|"Link,l=4"), (dim=63|id=789|"Link,l=3"))
[5] ((dim=4|id=774|"Electron,Site,n=5"), (dim=285|id=447|"Link,l=5"), (dim=166|id=133|"Link,l=4"))
[6] ((dim=4|id=998|"Electron,Site,n=6"), (dim=375|id=259|"Link,l=6"), (dim=285|id=447|"Link,l=5"))
[7] ((dim=4|id=324|"Electron,Site,n=7"), (dim=488|id=21|"Link,l=7"), (dim=375|id=259|"Link,l=6"))
[8] ((dim=4|id=506|"Electron,Site,n=8"), (dim=551|id=689|"Link,l=8"), (dim=488|id=21|"Link,l=7"))
[9] ((dim=4|id=606|"Electron,Site,n=9"), (dim=649|id=970|"Link,l=9"), (dim=551|id=689|"Link,l=8"))
[10] ((dim=4|id=196|"Electron,Site,n=10"), (dim=791|id=687|"Link,l=10"), (dim=649|id=970|"Link,l=9"))
[11] ((dim=4|id=298|"

In [99]:
memory_info_all_gpus()

Tesla V100-SXM2-32GB #1 memory usage: 59.25 % (18.96 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 92.84 % (29.71 GB/ 32.0 GB)


2-element Vector{Any}:
 59.25
 92.84